In [17]:
# reference
# https://www.youtube.com/watch?v=OSA5fZZwEW4

from IPython.display import display, clear_output
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import numpy as np
from os import listdir
import shutil
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import optim
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from os import listdir
import shutil
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch import optim
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt


class EllipseModel(nn.Module):
    def __init__(self, n, use_cuda=True):
        super(EllipseModel, self).__init__()
        self.num = n
        self.fr = 8  # Fourier terms
        self.use_cuda = use_cuda
        self.posx = torch.nn.Parameter(
            torch.randn((self.num), requires_grad=True))
        self.posy = torch.nn.Parameter(
            torch.randn((self.num), requires_grad=True))
        self.radius = torch.nn.Parameter(-3.0 *
                                         torch.randn((self.num), requires_grad=True))
        self.fweights_sin = torch.nn.Parameter(
            torch.zeros((self.fr, self.num), requires_grad=True))
        self.fweights_cos = torch.nn.Parameter(
            torch.zeros((self.fr, self.num), requires_grad=True))
        self.scale_x = torch.nn.Parameter(
            torch.randn((self.num), requires_grad=True))
        self.ecc2 = torch.nn.Parameter(
            torch.randn((self.num), requires_grad=True))
        self.colors = torch.nn.Parameter(
            torch.zeros((self.num, 3), requires_grad=True))
        self.rotate = torch.nn.Parameter(
            torch.zeros((self.num), requires_grad=True))
        self.order = torch.nn.Parameter(
            torch.ones((self.num), requires_grad=True))

    def forward(self, isize, t):
        posx = 3.0 * (self.posx.sigmoid()) - 1.0
        posy = 3.0 * (self.posy.sigmoid()) - 1.0
        radius = (self.radius.sigmoid())
        colors = (self.colors).tanh()
        # background = (self.background).sigmoid() + 0.5
        rotate = 3.141592 * self.rotate
        scale_x = (self.scale_x).sigmoid() * 4.0
        ecc2 = (self.ecc2).sigmoid() * 4.0
        order = self.order
        fr = self.fr  # Fourier terms

        X = torch.unsqueeze(torch.arange(
            0.5, isize + 0.5, 1) / isize, 0)
        Y = torch.unsqueeze(torch.arange(
            0.5, isize + 0.5, 1) / isize, 0)
        I = torch.unsqueeze(torch.ones(isize), 0)
        if self.use_cuda:
            X = X.cuda()
            Y = Y.cuda()
            I = I.cuda()

        X = X * I
        Y = torch.transpose(Y * I, 0, 1)
        X = X.unsqueeze_(0)
        Y = Y.unsqueeze_(0)

        X = X.expand(self.num, 1, isize, isize) - \
            posx.reshape(self.num, 1, 1, 1)
        Y = Y.expand(self.num, 1, isize, isize) - \
            posy.reshape(self.num, 1, 1, 1)
        X, Y = X * (rotate.reshape(self.num, 1, 1, 1).cos()) - Y * (rotate.reshape(self.num, 1, 1, 1,).sin()), \
            X * (rotate.reshape(self.num, 1, 1, 1).sin()) + \
            Y * (rotate.reshape(self.num, 1, 1, 1).cos())

        X = scale_x.reshape(self.num, 1, 1, 1) * X
        Y = ecc2.reshape(self.num, 1, 1, 1) * Y

        R = torch.sqrt(X.square() + Y.square())
        theta = -1.0 * (torch.acos(X / (R + 0.0001)) *
                        (1.0 * (Y > 0.0) - 1.0 * (Y < 0.0)))

        fourier = torch.arange(1, fr + 1).expand(self.num,
                                                 fr).permute(1, 0).reshape(fr, self.num, 1, 1, 1)

        if self.use_cuda:
            fourier = fourier.cuda()

        fourier = (fourier * (theta.expand(fr, self.num, 1, isize, isize)))
        fourier = (self.fweights_sin.reshape(fr, self.num, 1, 1, 1) * fourier.sin() +
                   self.fweights_cos.reshape(fr, self.num, 1, 1, 1) * fourier.cos()).sum(0)
        filter_values = (radius.reshape(self.num, 1, 1, 1) *
                         (1.0 + 0.1 * fourier) - R) / radius.reshape(self.num, 1, 1, 1)
        # filter_values = (radius.reshape(self.num, 1, 1, 1) - R)
        sharpness = 16.0
        filters_soft = torch.tanh(sharpness * filter_values)

        # weights = 1000.0*(1-torch.mean(1.0*(filters_soft > 0), (2, 3),
        #                 keepdim=True)) * order.reshape(self.num, 1, 1, 1)
        # filter_sort = (1-t)*torch.exp(np.log(t)*(torch.argsort(
        #     torch.argsort(weights*1.0*(filters_soft > 0), 0, descending=True), 0)))

        # filters2 = filter_sort * filters_soft * (filters_soft > 0.0)
        # filters2 = filters2 / (0.0001+torch.sum(filters2, 0, keepdim=True))
        filters2 = filters_soft * (filters_soft > 0.0)
        return (filters_soft, filters2, radius, colors)


In [18]:
isize = 256
img = Image.open("example_input_1.png").convert("RGB")
transform = transforms.Compose([
    transforms.Resize(isize),
    transforms.CenterCrop(isize),
    transforms.ToTensor()
])
img = transform(img)
img = img.cuda()
avg_pool = nn.AvgPool2d(21, stride=1, padding=10)
avg_img = avg_pool(img)
img_var = avg_pool((img - avg_img).square()).sqrt()
# plt.imsave("src.png", img)
num = 256
# model = WaveModel(num)
model = EllipseModel(num, use_cuda=True)
# try:
#     model.load_state_dict(torch.load('model.ckpt'))
# except:
#     pass
model = model.cuda()
optimizer = optim.AdamW(model.parameters(), lr=0.01)
out = None
num_epochs = 1000
try:
    shutil.rmtree('.scratch')
except:
    pass
os.mkdir('.scratch')
for epoch in range(0, num_epochs):
    torch.cuda.empty_cache()

    filters_soft, filters2, radius, colors = model(isize, 0.001)
    mrgb = torch.sum(img.expand(num, 3, isize, isize)*filters2*(filters_soft > 0.0), (2, 3),
                     keepdim=True)/(torch.sum(filters2*(filters_soft > 0.0), (2, 3), keepdim=True)+0.00001)
    rgb = colors.reshape((num, 3, 1, 1))*filters2.expand(num, 3, isize, isize)
    out = torch.sum(filters2*rgb, 0)
    # pic_loss = (img-out).abs().mean()
    # pic_loss = ((img_var*((img-out).square()))/img_var.mean()).mean()
    pic_loss = ((((img-out).square()))).mean().sqrt()
    # pic_loss = ((img_var*((img-out).square()))/img_var.mean()).mean()
    loss = pic_loss

    # grayscale, colors = model()
    # rgb = colors.reshape((model.num, 3, 1, 1))
    # out = torch.sum(rgb * grayscale, 0) / model.num
    # pic_loss = ((img - out).square()).mean().sqrt()
    # # pic_loss = ((img - out).abs()).mean()
    # loss = pic_loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    out = out.clamp(0.0, 1.0)
    plt.imsave(".scratch/out_" + str(epoch) + ".png", out.permute(1, 2, 0).detach().cpu().numpy())
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# plt.imsave("out.png", out.permute(1, 2, 0).detach().cpu().numpy())
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/1000], Loss: 0.5117
Epoch [2/1000], Loss: 0.3528
Epoch [3/1000], Loss: 0.2642
Epoch [4/1000], Loss: 0.2887
Epoch [5/1000], Loss: 0.3123
Epoch [6/1000], Loss: 0.2952
Epoch [7/1000], Loss: 0.2576
Epoch [8/1000], Loss: 0.2282
Epoch [9/1000], Loss: 0.2242
Epoch [10/1000], Loss: 0.2349
Epoch [11/1000], Loss: 0.2422
Epoch [12/1000], Loss: 0.2393
Epoch [13/1000], Loss: 0.2284
Epoch [14/1000], Loss: 0.2164
Epoch [15/1000], Loss: 0.2098
Epoch [16/1000], Loss: 0.2097
Epoch [17/1000], Loss: 0.2115
Epoch [18/1000], Loss: 0.2104
Epoch [19/1000], Loss: 0.2057
Epoch [20/1000], Loss: 0.2000
Epoch [21/1000], Loss: 0.1962
Epoch [22/1000], Loss: 0.1948
Epoch [23/1000], Loss: 0.1941
Epoch [24/1000], Loss: 0.1926
Epoch [25/1000], Loss: 0.1903
Epoch [26/1000], Loss: 0.1879
Epoch [27/1000], Loss: 0.1859
Epoch [28/1000], Loss: 0.1841
Epoch [29/1000], Loss: 0.1822
Epoch [30/1000], Loss: 0.1802
Epoch [31/1000], Loss: 0.1786
Epoch [32/1000], Loss: 0.1773
Epoch [33/1000], Loss: 0.1759
Epoch [34/1000], Lo

In [19]:
plt.imsave("experiment_1.png", out.permute(1, 2, 0).detach().cpu().numpy())

In [20]:
from moviepy.editor import *
import cv2
fourcc = cv2.VideoWriter_fourcc(*"H264")
video = cv2.VideoWriter("experiment_video_1.mp4", fourcc, 24, (isize, isize))
gen = {}
for n in range(0, num_epochs):
    image = cv2.imread(f".scratch/out_" + str(n) + ".png")
    video.write(image)
video.release()

In [4]:
torch.save(model.state_dict(), 'model.ckpt')

In [2]:

isize = 256
num = 1024
white = torch.tensor([1.0]).expand(num, 3, isize, isize)
model = EllipseModel(n=num, use_cuda=False)
model.load_state_dict(torch.load('model.ckpt'))
model = model.cpu()

filters_soft, filters2, radius, colors = model(isize, 0.001)
# mrgb = torch.sum(white*filters2*(filters_soft > 0.0), (2, 3),
#                     keepdim=True)/(torch.sum(filters2*(filters_soft > 0.0), (2, 3), keepdim=True)+0.00001)
rgb = colors.reshape((num, 3, 1, 1))*filters2.expand(num, 3, isize, isize)
out = torch.sum(filters2*rgb, 0)
out = out.clamp(0.0, 1.0)
plt.imsave("experiment.png", out.permute(1, 2, 0).detach().cpu().numpy())

In [121]:
from IPython.display import Video

Video("video.mp4")

In [100]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = 'max_split_size_mb:32'

class Model 